In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

In [2]:
def load_data(train_dir, test_dir, img_size=(640, 640), batch_size=32):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        label_mode='categorical',
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )
    
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        label_mode='categorical',
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False,
        seed=42
    )
    return train_ds, test_ds

In [3]:
def create_augmentation_layer():
    def add_salt_pepper(image):
        prob = 0.02
        rnd = tf.random.uniform(shape=tf.shape(image)[:-1])
        salt = tf.cast(rnd > (1 - prob/2), tf.float32) * 255.0
        pepper = tf.cast(rnd < prob/2, tf.float32) * (-255.0)
        return tf.clip_by_value(image + salt + pepper, 0.0, 255.0)

    return tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomBrightness(0.2),
        layers.Lambda(lambda x: tf.image.gaussian_filter2d(x, [3, 3], 1.0)),
        layers.Lambda(add_salt_pepper),
    ])

In [4]:
def build_model(num_classes=5):
    # Base model
    base_model = EfficientNetB0(
        include_top=False,
        input_shape=(640, 640, 3),
        weights='imagenet'
    )
    base_model.trainable = False

    # Custom head
    inputs = layers.Input(shape=(640, 640, 3))
    x = create_augmentation_layer()(inputs)
    x = EfficientNetB0.preprocess_input(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs, outputs)

In [5]:
def train_model(model, train_ds, test_ds, epochs=50):
    callbacks = [
        ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy'),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    history = model.fit(
        train_ds,
        validation_data=test_ds,
        epochs=epochs,
        callbacks=callbacks
    )
    return history

In [6]:
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.show()

In [8]:
# Parameters
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
IMG_SIZE = (640, 640)
BATCH_SIZE = 32
EPOCHS = 50

# Load data
train_ds, test_ds = load_data(TRAIN_DIR, TEST_DIR, IMG_SIZE, BATCH_SIZE)

# Build model
model = build_model()

# Train model
history = train_model(model, train_ds, test_ds, EPOCHS)

# Plot training history
plot_history(history)

# Evaluate model
test_loss, test_acc = model.evaluate(test_ds)
print(f'\nTest Accuracy: {test_acc*100:.2f}%')

# Save final model
model.save('acne_classifier_final.h5')

Found 3159 files belonging to 5 classes.
Found 127 files belonging to 5 classes.


NotImplementedError: Exception encountered when calling Sequential.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Sequential.call():
  • args=('<KerasTensor shape=(None, 640, 640, 3), dtype=float32, sparse=False, name=keras_tensor_488>',)
  • kwargs={'mask': 'None'}